## CLASSIFICATION EXAMPLE

### -Jannadi Makrem-

##### We'll be working with some California Census Data, we'll be trying to use various features of an individual to predict what class of income they belong in (>50k or <=50k).

#### The Data:

In [8]:
import pandas as pd

In [10]:
census = pd.read_csv("C:/Users/DELL/Tensorflow_Project/census_data.csv")

In [11]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


#### TensorFlow won't be able to understand strings as labels, so we will convert the Label column to 0s and 1s instead of strings.

In [12]:
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [13]:
def label_fix(label):
    if label==' <=50K':
        return 0
    else:
        return 1

In [14]:
census['income_bracket'] = census['income_bracket'].apply(label_fix)

In [15]:
census['income_bracket'].unique()

array([0, 1], dtype=int64)

### Perform a Train Test Split on the Data

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
x_data = census.drop('income_bracket',axis=1)
y_labels = census['income_bracket']
X_train, X_test, y_train, y_test = train_test_split(x_data,y_labels,
                                                    test_size=0.3,
                                                    random_state=101)

### Create the Feature Columns for tf.esitmator

##### Take note of categorical vs continuous values!

In [18]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

##### Import Tensorflow

In [19]:
import tensorflow as tf

##### Create the tf.feature_columns for the categorical values

In [20]:
gender = tf.feature_column.categorical_column_with_vocabulary_list(
    "gender",["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket(
    "occupation",hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket(
    "marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket(
    "relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket(
    "education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket(
    "workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket(
    "native_country", hash_bucket_size=1000)

##### Create the continuous feature_columns for the continuous values using numeric_column

In [21]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

##### Put all these variables into a single list with the variable name feat_cols

In [22]:
feat_cols = [gender,occupation,marital_status,relationship,education,workclass,
             native_country,age,education_num,capital_gain,capital_loss,
             hours_per_week]

### Create Input Function

In [23]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,
                                                 batch_size=100,
                                                 num_epochs=None,
                                                 shuffle=True)

##### Create the model with tf.estimator (LinearClassifier)

In [24]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\DELL\\AppData\\Local\\Temp\\tmphd39etm0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000002A2ACC6E10>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


##### Train the model on the Data, for 5000 steps

In [25]:
model.train(input_fn = input_func,steps=5000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\DELL\AppData\Local\Temp\tmphd39etm0\model.ckpt.
INFO:tensorflow:loss = 69.3147, step = 1
INFO:tensorflow:global_step/sec: 168.08
INFO:tensorflow:loss = 343.425, step = 101 (0.599 sec)
INFO:tensorflow:global_step/sec: 281.338
INFO:tensorflow:loss = 113.153, step = 201 (0.355 sec)
INFO:tensorflow:global_step/sec: 254.424
INFO:tensorflow:loss = 78.6489, step = 301 (0.394 sec)
INFO:tensorflow:global_step/sec: 264.268
INFO:tensorflow:loss = 60.7092, step = 401 (0.378 sec)
INFO:tensorflow:global_step/sec: 285.362
INFO:tensorflow:loss = 56.0403, step = 501 (0.350 sec)
INFO:tensorflow:global_step/sec: 295.071
INFO:tensorflow:loss = 91.9415, step = 601 (0.339 sec)
INFO:tensorflow:global_step/sec: 300.854
INFO:tensorflow:loss = 27.7362, step = 701 (0.332 sec)
INFO:tensorflow:global_step/sec: 284.953
INFO:tensorflow:loss = 202.947, step = 801 (0.351 sec)
INFO:tensorflow:global_step/sec: 286.276
INFO

### Evaluation

##### Create a prediction input function

In [26]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),
                                              shuffle=False)

##### We will use model.predict() and pass in the input function. This will produce a generator of predictions, which we can then transform into a list, with list()

In [27]:
predictions = list(model.predict(input_fn=pred_fn))

INFO:tensorflow:Restoring parameters from C:\Users\DELL\AppData\Local\Temp\tmphd39etm0\model.ckpt-5000


##### Each item in the list will look like this:

In [28]:
predictions[0]

{'class_ids': array([0], dtype=int64),
 'classes': array([b'0'], dtype=object),
 'logistic': array([ 0.37067723], dtype=float32),
 'logits': array([-0.52931267], dtype=float32),
 'probabilities': array([ 0.62932283,  0.3706772 ], dtype=float32)}

##### Create a list of only the class_ids key values from the prediction list of dictionaries, these are the predictions we will use to compare against the real y_test values.

In [29]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [30]:
final_preds[:10]

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

##### Import classification_report from sklearn.metrics and then see if we can figure out how to use it to easily get a full report of our model's performance on the test data

In [31]:
from sklearn.metrics import classification_report


In [52]:
print(classification_report(y_test,final_preds))

             precision    recall  f1-score   support

          0       0.88      0.93      0.91      7436
          1       0.73      0.60      0.66      2333

avg / total       0.85      0.85      0.85      9769

